In [1]:
# tensorflow에 대해서 알아보자!

import tensorflow as tf

# 상수 node를 만든다.
# node인데 상수 값을 가지고 있는 node
node1 = tf.constant(10, dtype=tf.float32)
node2 = tf.constant(30, dtype=tf.float32)

node3 = node1 + node2

# Session이라는 게 필요하다! => 그래프를 실행시키기 위해서!
# 그래프를 실행시켜주는 주체
sess = tf.Session()

# 노드를 실행해야 노드가 가지고 있는 값이 나온다.
print(sess.run(node1))   # graph를 실행시킨다! -> node1을 실행

print(sess.run(node3))   # graph를 실행시킨다! -> node3을 실행
# node3는 node1과 node2가 연결되어 있어서
# node3을 실행시키면 node1, node2가 같이 실행된다.
# 그리고 결과 값으로 node1 + node2인 40을 출력

print(sess.run([node1, node3]))
# 여러개의 노드를 실행시키려면 node를 list로 묶는다.



10.0
40.0
[10.0, 40.0]


In [3]:
# 현재 tensorflow 버전은 1.15 버전이다!

import tensorflow as tf

node1 = tf.placeholder(dtype=tf.float32)   # scalar
# shape을 명시해주지 않으면 scalar를 대상으로 한다.
# 만약 다차원데이터가 입력되려면 shape을 지정해줘야 한다.

node2 = tf.placeholder(dtype=tf.float32)

node3 = node1 + node2

sess = tf.Session()

# 먹이를 주지 않고 실행하면 에러가 난다.
# node1과 node2의 값이 없기 때문
# 그래서 dictionary 형태로 먹이를 준다.
result = sess.run(node3, feed_dict={node1: 10,
                                    node2: 30})
print(result)

40.0


In [7]:
# Q1, Q2, Q3, Exam 예제를 이용해서
# Tensorflow를 사용해서 Multiple Linear Regression을 구현해보자!
import tensorflow as tf
import pandas as pd
import numpy as np

# Raq Data Loading
df = pd.read_csv('../data/student_exam_score.csv')
# display(df.head())

# 결측치 없고, 이상치 없고, 정규화 필요 없다.
# 데이터 전처리 안한다.

# Training Data Set
x_data = df.drop('exam', axis=1, inplace=False)   # (25, 3)
t_data = df['exam'].values.reshape(-1,1)   # (25, 1)

# Prediction
predict_data = np.array([[90, 100, 95]])   # (1, 3)

# Placeholder
X = tf.placeholder(shape=[None,3], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)
# shape을 명시할 때 ()가 아닌 []를 사용한다.

# Weight, bias
# 변수노드 => 안의 값이 계속 변할 수 있는 노드
# W는 어떤 형태여야 하나요? => (3, 1) 형태여야 하고 랜덤하게 생성
W = tf.Variable(tf.random.normal([3, 1]))   # 표준정규분포에서 난수를 추출
b = tf.Variable(tf.random.normal([1]))

# model(hypothesis, 가설, predict model)   # y = Wx + b  => XW + b
H = tf.matmul(X, W) + b   # Tensorflow가 제공하는 행렬 곱 함수

# loss function
loss = tf.reduce_mean(tf.square(H - T))   # 평균제곱오차

# train node를 생성해야 한다!
train = tf.train.GradientDescentOptimizer(learning_rate=1e-7).minimize(loss)
# GradientDescent 알고리즘을 진행하는 주체를 만든다
# learning_rate를 인자로 주고 minimize 메서드에 편미분할 노드를 인자로 넣는다.
# 학습을 단 1번 실행한다.

# Session을 생성하고 초기화를 진행
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 초기화 구문.
# (그래프를 실행하기 전에 먼저 실행되어야 한다.)
# tf.Variable을 사용하면 반드시 초기화를 해줘야 한다.

# 반복 학습
for step in range(300000):
    
    # Placeholder로 선언된 값에 먹이를 줘야한다.
    _, W_val, b_val, loss_val = sess.run([train, W, b, loss],
                                         feed_dict={X: x_data,
                                                    T: t_data})
    if step % 30000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val, b_val, loss_val))
        
        

W : [[ 0.4404185 ]
 [-0.29963538]
 [ 0.05776295]], b : [0.07889124], loss : 22021.984375
W : [[0.9928354]
 [0.3152069]
 [0.7173451]], b : [0.08618753], loss : 14.133596420288086
W : [[0.94643795]
 [0.3245982 ]
 [0.7532662 ]], b : [0.08596402], loss : 12.955582618713379
W : [[0.90339094]
 [0.33410728]
 [0.7858133 ]], b : [0.0857405], loss : 11.954911231994629
W : [[0.86343884]
 [0.34369418]
 [0.8152794 ]], b : [0.08551698], loss : 11.103924751281738
W : [[0.82657564]
 [0.35299972]
 [0.8420019 ]], b : [0.08529346], loss : 10.382430076599121
W : [[0.7924041 ]
 [0.3621382 ]
 [0.86629754]], b : [0.08506995], loss : 9.76748275756836
W : [[0.76078975]
 [0.37093526]
 [0.8884322 ]], b : [0.08484643], loss : 9.243639945983887
W : [[0.7314452 ]
 [0.37950936]
 [0.9085687 ]], b : [0.08462291], loss : 8.796031951904297
W : [[0.70417684]
 [0.38790902]
 [0.92685694]], b : [0.08439939], loss : 8.413019180297852


In [8]:
# Prediction

result = sess.run(H,
                  feed_dict={X: np.array([[89, 100, 95]])})
print(result)

[[189.7412]]
